## Лабораторная работа 5. Pipelines. Feature Selection. Multilabel classification.

Постарайтесь оформить всю работу как один монолитный рассказ/отчет. Избегайте дублирования кода. Избегайте использования циклов, вместо этого ищите готовый алгоритм в пакетах. Подписывайте все графики, а также их оси, если график содержит несколько кривых, то обязательно пользуйтесь легендой. Также неукоснительно соблюдайте PEP8. За несоблюдение этих простейших правил итоговые баллы могут быть снижены безапелляционно.

### 1. (20 баллов) Pipelines. Binary Relevance

![](./static/pipeline-2.png)

В этой лабораторной работе вы будете работать с данными из конкурса [Competition 3, Yandex SHAD, Spring 2018](https://www.kaggle.com/c/competition-3-yandex-shad-spring-2018). Описание данных и [baseline решение](http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/l3icnmywjijx3i1/baseline_competition_3.ipynb) можно найти на странице соревнования.

In [ ]:
import lingvox as lx
from lingvox import *

In [ ]:
import warnings
warnings.filterwarnings("ignore")

**1. (1 балл)** Подготовьте данные: в этом задании и далее вы будете оценивать качество (метрика качества: `f1-score`, усредненная по всем объектам) при помощи кросс-валидации на 3 фолда, предварительно перемешав данные при помощи функции `shuffle` с `random_state=1120`. Подготовьте 4 типа независимых `Pipelines`: 
* бинаризированные данные, 
* tf-idf, 
* tf, 
* логарифм tf. 

Предварительно отберите признаки с ненулевой дисперсией.

<font color="purple">_1.1 &emsp; Данные_</font>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
X_train_raw = lx.read_data_from_file("./data/X_train.csv", (15000, 60000)).astype(float)
y_train = lx.read_labels_from_file("./data/Y_train.csv", (15000, 100))
# отбираем признаки с ненулевой дисперсией
X_train = X_train_raw[:, np.where(np.sum(X_train_raw, axis=0) > 0)[1]]

<font color="purple">_1.2 &emsp; Пайплайны_</font>

In [ ]:
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from scipy import sparse

In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=1120)
binarizer = Binarizer()
tfidf = TfidfTransformer()

**2. (4 балла)** Подберите параметры `LogisticRegression` (тип регуляризации `penalty` и коэффициент регуляризации `C`, с фиксированным параметром `class_weight="balanced"`), оптимизируя метрику качества `make_scorer(f1_score, average="samples")`. Используйте подход **one-vs-rest** при помощи класса `OneVsRestClassifier`. Проведите эксперименты для четырех типов исходных данных из прошлого задания. Визуализируйте полученные результаты. Сделайте краткое резюме.

In [ ]:
OneVsRestClassifier


In [ ]:
param_logreg = {
    "penalty": ["l1", "l2"],
    "C": np.logspace(-3, 3)
}

In [ ]:
logreg_proto = OneVsRestClassifier(LogisticRegression(class_weight="balanced"))
logreg = GridSearchCV(
    param_grid=param_logreg,
    metric=make_scorer(f1_score, average="samples"),
    cv=kf,
    verbose=1,
    n_jobs=-1
)

**3. (2 балла)** Объедените два лучших набора данных в один при помощи класса `FeatureUnion`. Подберите параметры для классификатора из прошлого пункта на полученных объединением признаках. Сравните результаты с предыдущим заданием. Визуализируйте результаты. Сделайте краткое заключение.

**4. (5 баллов)**
Отберите бинаризированные признаки при помощи класса `SelectKBest` и $\chi^2$-статистики (вам понадобится функция `chi2` и класс `FunctionTransformer`). Умножьте каждый отобранный признак на значение $\chi^2$-статистики, соответствующей этому признаку. В качестве алгоритма классификации на полученных признаках используйте класс `KNeighborsClassifier` с параметрами: `n_neighbors=1`, `metric="cosine"`, `weights="distance"`. Проведите эксперименты для различных значений параметра `k` класса `SelectKBest`. Также сравните результаты для взвещенных признаков при помощи $\chi^2$-статистики и на исходных бинаризированных признаках. Визуализируйте результаты. Сделайте краткое заключение. Ответьте на вопросы. В каких вариациях "существует" алгоритм метода ближайщего соседа для задачи multi-label классификации? Какую из "версий" вы используете в этом задании? 

**Про метрику хи-квадрат ($\chi^2$)** 

$\chi^2$ — тест на зависимость бинарной метки от данного признака. $\chi^2$ показывает, насколько сильно отличается значение в нулевом классе и в первом классе (можно и для n классов). Чем больше $\chi^2$ — тем больше различия между классами по данному признаку, тем лучше данный признак позволяет отличить один класс от другого. </font>

**5. (4 балла)** Разберитесь в алгоритме, который реализует класс [ClassifierChain](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.ClassifierChain.html). Опишите кратко принцип работы этого алгоритма. Проведите эксперименты для логистической регрессии и набора данных из _задания 3_: при фиксированном параметре `penalty="l2"` подберите наилучший коэффициент регуляризации `C`. Визуализируйте результаты. Сравните результаты с результатами задания 3. Сделайте краткое заключение.

**6. (4 балла)** Так как метрика качества `f1-score` оперирует метками категорий, а не вероятностями принадлежности к категориям, то такая метрика качества чувствительна к выбору пороговой вероятности классификатора, предсказывающего вероятности принадлежности к каждой из категорий. Для лучших параметров _задания 5_ подберите наилучший ("глобальный") порог для вероятности принадлежности к категории, используя две стратегии: переберите порог для исходных вероятностей из _задания 5_; переберите порог для "трансформированных" вероятностей из _задания 5_ (для каждого объекта разделите вектор вероятностей на максимальное значение вероятности в векторе вероятностей). Визуализируйте результаты. Сделайте краткое заключение.

### 2. (10 баллов) Pipelines. Label Powerset

**7. (4 балла)** В файле `lingvox.py` представлен класс `MLScoreWrapper`, позволяющий решать задачу multi-label классификации в label-powerset постановке и определять качество именно для задачи в постановке multi-label. Разберитесь в работе этого класса. Реализуйте метод `ml2mc` по аналогии с методом `mc2ml`. Протестируйте класс на основную функциональность (в текущей реализации есть `bug`, однако для наших данных он не проявляется). 

**8. (4 балла)** Перейдите от multi-label задачи к multi-class (label-powerset) задаче. Используйте для этого класс `MLScoreWrapper`. Для логистической регрессии при фиксированных параметрах `penalty="l2"`, `C=0.1`, `multi_class="ovr"` подберите параметр `class_weight`. Визуализируйте результаты. Сделайте краткое заключение. 

**9. (2 балла)** Ответьте на вопросы. В чем преимущества и недостатки подхода label powerset от подхода binary relevance в задачах multi-label классификации? Какие алгоритмы в пакете sklearn могут "из коробки" работать с multi-label задачами, с multi-class задачами?

### 3. (15 баллов) Бонусное задание

**10. (5 баллов)** Протестируйте функции записи финального `submission`. Напишите корректнкую версию. Объясните в чем ошибка в текущей реализации.

In [ ]:
import numpy as np


def pred_to_submission(y_pred):
    def transform(x):
        x = [str(x) for x in np.nonzero(x)[0] + 1]
        return " ".join(x)

    submission = pd.DataFrame()
    submission["Id"] = list(range(1, len(y_pred) + 1))
    submission["Labels"] = np.apply_along_axis(transform, axis=1, arr=y_pred)

    return submission

**11. (15 баллов)** 
Сравните результаты работы метода ближайщего соседа в постановках: `MLkNN`, `BRkNN`, `LPkNN`. Постройте `boxplot` для каждого из алгоритмов. Также кратко объясните принципы и отличия этих алгоритмов. Для проведения экспериментов используйте библиотеку [scikit-multilearn](http://scikit.ml/). Подберите наилучшие параметры `n_neighbors` у каждого из подходов при фиксированных параметрах `weights="distance"`, `metric="cosine"`. Используйте любой из понравившихся наборов признаков. (Одна из сложностей этого задания заключается в реализациях алгоритмов в библиотеке scikit-multilearn, ни один из этих алгоритмов "из коробки" не поддерживает параметры `weights`, `cosine` и опционально полезный параметр `n_jobs`. После установки пакета вам потребуется частично изменить исходный код для классов, реализующих эти алгоритмы, так, чтобы они поддерживали вышеперечисленные параметры и были совместимы по api с sklearn. 

В качестве решения этого задания предполагается проведение не только экспериментов, но и отчет в виде скринов "переделанного" кода).  